In [13]:
# imports
from utils import *

In [ ]:
import json

dataset_path = "data/lean_workbook.json"
with open(dataset_path, "r") as f:
    dataset = json.load(f)
print(type(dataset))
print(len(dataset))
print(dataset[0].keys())
print(dataset[0]["proof"])
proof_problems = [sample for sample in dataset if len(sample["proof"]) > 0]
print(len(proof_problems))

In [5]:
print(json.dumps(dataset[0], indent=2))
print(json.dumps(proof_problems[1], indent=2))

{
  "natural_language_statement": "Let $a,b,c$ be positive real numbers .Prove that $ \\frac{b+c}{\\sqrt{a^2 + 8bc}} + \\frac{c+a}{\\sqrt{b^2 + 8ca}} + \\frac{a+b}{\\sqrt{c^2 + 8ab}} \\geq 2. $",
  "answer": "2",
  "tags": [
    "inequality"
  ],
  "formal_statement": "theorem lean_workbook_0 (a b c : \u211d) (ha : 0 < a) (hb : 0 < b) (hc : 0 < c) : (b + c) / Real.sqrt (a ^ 2 + 8 * b * c) + (c + a) / Real.sqrt (b ^ 2 + 8 * c * a) + (a + b) / Real.sqrt (c ^ 2 + 8 * a * b) \u2265 2  :=  by sorry",
  "split": "lean_workbook",
  "proof": []
}
{
  "natural_language_statement": "Prove that $x-1 \\geq \\ln x$ for $x>0$.",
  "answer": "",
  "tags": [
    "inequality",
    "logarithm",
    "calculus"
  ],
  "formal_statement": "theorem lean_workbook_26 (x : \u211d) (hx : 0 < x) : x - 1 \u2265 Real.log x  :=  by sorry",
  "split": "lean_workbook",
  "proof": [
    "have h1 : 0 \u2264 (x - 1)^2 := sq_nonneg (x - 1)\nnlinarith [log_le_sub_one_of_pos hx]",
    "nlinarith [log_le_sub_one_of_pos hx]"

In [ ]:
from datasets import load_dataset
dataset = load_dataest("hoskinson-center/proofnet")  # lean4 version of proofnet

In [1]:
# download MATH
from datasets import load_dataset
dataset = load_dataset("hendrycks/competition_math")["train"]
print(dataset)
print(type(list(dataset)))
print(list(dataset)[0])

Dataset({
    features: ['problem', 'level', 'type', 'solution'],
    num_rows: 7500
})
<class 'list'>
{'problem': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).', 'level': 'Level 5', 'type': 'Algebra', 'solution': 'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.'}


In [1]:
from processing import build_library_from_outputs
output_path = "outputs/qwen_MATH_lemma_extract.jsonl"
lemmas = build_library_from_outputs(output_path)

Found 7500 samples
Extracted 4505 lemmas


In [2]:
from utils import dump_jsonl
from dataclasses import asdict
save_path = "libraries/qwen_lemmas.jsonl"
dump_jsonl([asdict(lemma) for lemma in lemmas], save_path)

In [7]:
# extract lemma requests
from utils import dump_jsonl
from processing import extract_relevant_lemmas_from_outputs
outputs_path = "outputs/test_lemma_descriptions.jsonl"
samples = extract_relevant_lemmas_from_outputs(outputs_path)
samples = [s for s in samples if s["lemma_requests"] is not None and len(s["extracted_lemma_requests"]) > 0]
dump_jsonl(samples, "outputs/test_extracted_lemma_descriptions.jsonl")

Found 1552 samples
Found 6795 requests


In [6]:
# pair samples by computing pairwise cosine similarities between 
from library import compute_batched_similarity
from utils import load_jsonl
import torch
sources = load_jsonl("outputs/test_extracted_lemma_descriptions_embeddings.jsonl")
targets = load_jsonl("libraries/qwen_lemmas_embeddings.jsonl")

In [3]:
print(sources[0].keys())
print(sources[0]["extracted_lemma_requests"])
print(len(sources))
print(len(targets))

dict_keys(['problem', 'level', 'type', 'solution', 'lemma_requests', 'extracted_lemma_requests', 'embeddings'])
['\nA rational function \\( y = \\frac{P(x)}{Q(x)} \\) has vertical asymptotes at the values of \\( x \\) that make the denominator \\( Q(x) \\) zero, provided those values are not also roots of the numerator \\( P(x) \\).\n\n', '\nTo find the vertical asymptotes of a rational function, we need to solve the equation \\( Q(x) = 0 \\) for \\( x \\).\n\n', '\nThe quadratic equation \\( ax^2 + bx + c = 0 \\) can be factored or solved using the quadratic formula \\( x = \\frac{-b \\pm \\sqrt{b^2 - 4ac}}{2a} \\).\n\n', '\nIf a quadratic polynomial \\( Q(x) \\) can be factored into linear factors, the solutions to \\( Q(x) = 0 \\) are the roots of these linear factors.\n\n', "\nFor the function \\( y = \\frac{2}{x^2 + x - 6} \\), the denominator \\( x^2 + x - 6 \\) can be factored, and the solutions to the factored equation will give the vertical asymptotes.\n\nNow, let's apply thes

In [4]:
sources_embeddings = torch.tensor([embedding for sample in sources for embedding in sample["embeddings"]])
targets_embeddings = torch.tensor([sample["embedding"] for sample in targets])
sims_dict = compute_batched_similarity(sources_embeddings, targets_embeddings, batch_size=128)

100%|██████████| 151/151 [01:06<00:00,  2.28it/s]


In [5]:
print(len(sims_dict))
print(len(list(sims_dict.values())[0]))
sims = list(sims_dict.values())
import numpy as np
print(np.mean(sims[0]))
avg_sims = [np.mean(sim) for sim in sims]
max_sims = [np.max(sim) for sim in sims]
print(np.mean(avg_sims))
print(np.std(avg_sims))
print(np.mean(max_sims))

6795
4505
0.20641606
0.26650327
0.08968528
0.5877541


In [14]:
# find top three most similar lemmas to proof descriptions
from tqdm import tqdm
cnt = 0
for source in tqdm(sources):
    # retrieve most similar lemma for each source description
    source_sims = np.array([sims_dict[cnt+i] for i in range(len(source["extracted_lemma_requests"]))])
    max_source_sims = np.argmax(source_sims, axis=-1).flatten()
    source["retrieved_lemmas"] = [{
                                    "statement": targets[ind]["statement"],
                                    "proof": targets[ind]["proof"]
                                  } for ind in max_source_sims]
    cnt += len(source["extracted_lemma_requests"])

  0%|          | 0/1359 [00:00<?, ?it/s]

100%|██████████| 1359/1359 [01:35<00:00, 14.28it/s]


In [15]:
print(sources[0]["problem"])
print(sources[0]["retrieved_lemmas"])
print(len(sources[0]["retrieved_lemmas"]))

How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have?
[{'statement': 'A function will have a vertical asymptote where the denominator equals zero and the degree of that root is greater than the degree of the same root in the numerator.', 'proof': 'Consider a rational function \\( f(x) = \\frac{P(x)}{Q(x)} \\), where \\( P(x) \\) and \\( Q(x) \\) are polynomials. A vertical asymptote occurs at values of \\( x \\) where \\( Q(x) = 0 \\) and \\( P(x) \\neq 0 \\). This is because as \\( x \\) approaches these values, the denominator approaches zero while the numerator remains non-zero, causing the function to approach infinity or negative infinity.'}, {'statement': 'The vertical asymptote of a rational function \\( \\frac{P(x)}{Q(x)} \\) occurs where the denominator \\( Q(x) \\) is zero, provided \\( P(x) \\) is not also zero at that point.', 'proof': 'This follows directly from the definition of vertical asymptotes. As \\( x \\) approaches the roots of \\( Q(x) \\), t

In [22]:
import json
for sample in sources:
    sample["retrieved_lemmas_str"] = json.dumps(sample["retrieved_lemmas"], indent=2)
    sample.pop("embeddings")
print(sources[0]["retrieved_lemmas_str"])

[
  {
    "statement": "A function will have a vertical asymptote where the denominator equals zero and the degree of that root is greater than the degree of the same root in the numerator.",
    "proof": "Consider a rational function \\( f(x) = \\frac{P(x)}{Q(x)} \\), where \\( P(x) \\) and \\( Q(x) \\) are polynomials. A vertical asymptote occurs at values of \\( x \\) where \\( Q(x) = 0 \\) and \\( P(x) \\neq 0 \\). This is because as \\( x \\) approaches these values, the denominator approaches zero while the numerator remains non-zero, causing the function to approach infinity or negative infinity."
  },
  {
    "statement": "The vertical asymptote of a rational function \\( \\frac{P(x)}{Q(x)} \\) occurs where the denominator \\( Q(x) \\) is zero, provided \\( P(x) \\) is not also zero at that point.",
    "proof": "This follows directly from the definition of vertical asymptotes. As \\( x \\) approaches the roots of \\( Q(x) \\), the denominator approaches zero, causing the funct

In [23]:
from utils import dump_jsonl
dump_jsonl(sources, "inputs/MATH_test_retrieved_lemmas.jsonl")